In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import glob
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import TomekLinks 
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.model_selection import RepeatedStratifiedKFold
import datetime

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
filePathTrain = "C:/Users/athar/Downloads/encodedDataTrainFitX4.csv"
filePathTest = "C:/Users/athar/Downloads/encodedDataTestFitX4.csv"
trainData = pd.read_csv(filePathTrain)
testData = pd.read_csv(filePathTest)

In [4]:
# trainData["AOb"] = pd.to_datetime(trainData["Ob"], infer_datetime_format=True).dt.time
# trainData[['h', 'm', 's']] = trainData["AOb"].astype(str).str.split(':', expand=True).astype(int) #pd.DataFrame([(x.hour, x.minute, x.second)])
# trainData["totalMinutes"] = trainData["m"] + trainData["h"] * 60
# trainData = trainData.drop(columns=["AOb", "h", "m", "s"], axis=1)

In [5]:
# testData["AOb"] = pd.to_datetime(testData["Ob"], infer_datetime_format=True).dt.time
# testData[['h', 'm', 's']] = testData["AOb"].astype(str).str.split(':', expand=True).astype(int) #pd.DataFrame([(x.hour, x.minute, x.second)])
# testData["totalMinutes"] = testData["m"] + testData["h"] * 60
# testData = testData.drop(columns=["AOb", "h", "m", "s"], axis=1)

In [6]:
trainData["Ob"] = pd.to_datetime(trainData["Ob"], infer_datetime_format=True)

In [7]:
springStart = datetime.datetime(2021, 3, 1)
springEnd = datetime.datetime(2021, 5, 31, 23, 59, 59)
springDf = trainData[(trainData["Ob"] >= springStart) & (trainData["Ob"] <= springEnd)]

In [8]:
summer1Start = datetime.datetime(2021, 6, 1)
summer1End = datetime.datetime(2021, 7, 15, 23, 59, 59)

summer1Df = trainData[(trainData["Ob"] >= summer1Start) & (trainData["Ob"] <= summer1End)]
summer1Df.shape

(894420, 67)

In [9]:
summer2Start = datetime.datetime(2021, 7, 16)
summer2End = datetime.datetime(2021, 8, 31, 23, 59, 59)

summer2Df = trainData[(trainData["Ob"] >= summer2Start) & (trainData["Ob"] <= summer2End)]
summer2Df.shape

(1025717, 67)

In [10]:
fall1Start = datetime.datetime(2021, 9, 1)
fall1End = datetime.datetime(2021, 10, 15, 23, 59, 59)

fall1Df = trainData[(trainData["Ob"] >= fall1Start) & (trainData["Ob"] <= fall1End)]
fall1Df.shape

(611694, 67)

In [11]:
fall2Start = datetime.datetime(2021, 10, 16)
fall2End = datetime.datetime(2021, 11, 30, 23, 59, 59)
# springDf = trainData.loc[springStart:springEnd]
fall2Df = trainData[(trainData["Ob"] >= fall2Start) & (trainData["Ob"] <= fall2End)]
fall2Df.shape

(678783, 67)

In [12]:
winterStart = datetime.datetime(2021, 12, 1)
winterEnd = datetime.datetime(2021, 12, 31, 23, 59, 59)
# springDf = trainData.loc[springStart:springEnd]
winterDf = trainData[(trainData["Ob"] >= winterStart) & (trainData["Ob"] <= winterEnd)]

In [13]:
winter2Start = datetime.datetime(2021, 1, 1)
winter2End = datetime.datetime(2021, 2, 28, 23, 59, 59)
# springDf = trainData.loc[springStart:springEnd]
winter2Df = trainData[(trainData["Ob"] >= winter2Start) & (trainData["Ob"] <= winter2End)]
winterDf = pd.concat([winterDf, winter2Df], ignore_index=True)

In [14]:
springDf.to_csv("C:/Users/athar/Downloads/spring.csv", index=False)
winterDf.to_csv("C:/Users/athar/Downloads/winter.csv", index=False)
summer1Df.to_csv("C:/Users/athar/Downloads/summer1.csv", index=False)
summer2Df.to_csv("C:/Users/athar/Downloads/summer2.csv", index=False)
fall1Df.to_csv("C:/Users/athar/Downloads/fall1.csv", index=False)
fall2Df.to_csv("C:/Users/athar/Downloads/fall2.csv", index=False)
testData.to_csv("C:/Users/athar/Downloads/testNormEncoded.csv", index=False)

In [15]:
print(springDf.head())

                    Ob     value  target  R_flag  I_flag  Z_flag  B_flag  \
34 2021-03-01 09:30:00  0.917130   False       2      -1       0       1   
35 2021-03-01 10:30:00  0.920841   False       2      -1       0       1   
36 2021-03-01 10:46:00  0.912183   False       2      -1       2       0   
37 2021-03-01 10:47:00  0.909091   False       2      -1       2       0   
38 2021-03-01 10:48:00  0.905999   False       2      -1       2       0   

    measure_blackglobetemp  measure_impact  measure_leafwetness  ...  \
34                       0               0                    0  ...   
35                       0               0                    0  ...   
36                       0               0                    0  ...   
37                       0               0                    0  ...   
38                       0               0                    0  ...   

    station_SILR  station_SPIN  station_SPRU  station_TAYL  station_UNCA  \
34             0             0    

In [16]:
print(testData.head())

                Ob     value  R_flag  I_flag  Z_flag  B_flag  \
0  3/25/2021 10:37  0.931973       2      -1       2       0   
1  7/11/2021 13:22  0.975263       2      -1       0       1   
2  7/11/2021 13:23  0.975263       2      -1       0       1   
3  7/11/2021 13:24  0.975263       2      -1       0       1   
4  7/11/2021 13:25  0.975881       2      -1       0       1   

   measure_blackglobetemp  measure_impact  measure_leafwetness  measure_par  \
0                       0               0                    0            0   
1                       0               0                    0            0   
2                       0               0                    0            0   
3                       0               0                    0            0   
4                       0               0                    0            0   

   ...  station_SILR  station_SPIN  station_SPRU  station_TAYL  station_UNCA  \
0  ...             0             0             0            